In [4]:
from functions import get_points, write_to_laz, find_plane, find_clusters, kmeans_clusters, recursive_planes, filter_distance, hausdorff_distance, get_relevant_cids

import pandas as pd
from pyntcloud import PyntCloud
import numpy as np
import pdal
import shapely.wkt
from numpy.lib import recfunctions as rfn

import numpy as np
import pandas as pd
import matplotlib
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pyntcloud import PyntCloud
from sklearn.cluster import DBSCAN

from scipy.spatial import cKDTree

from pyntcloud.ransac.fitters import single_fit
from pyntcloud.ransac.models import RansacPlane

%load_ext autoreload
%autoreload 2

In [5]:
# test_area = [117813.6,487266.1,117942.1,487370.7]
# xmin,ymin,xmax,ymax = test_area

path_a10_2018 = '/var/data/rws/data/2018/entwined/ept.json'
path_a10_2019 = '/var/data/rws/data/2019/amsterdam_entwined/ept.json'

# random area: 
# wkt = 'POLYGON((117813.6 487266.1,117813.6 487370.7,117942.1 487370.7,117942.1 487266.1,117813.6 487266.1))'

# small area: 
# wkt = 'POLYGON((117813 487266,117813 487270,117815 487270,117815 487266,117813 487266))' # small! 

# portal:
# wkt = 'POLYGON((118032.6 488782.8,118032.6 488793.5,118078.8 488793.5,118078.8 488782.8,118032.6 488782.8))' # portal

# portal zonder palen
wkt = 'POLYGON((118037.0 488782.8,118037.0 488793.5,118072.0 488793.5,118072.0 488782.8,118037.0 488782.8))' # portal zonder palen

# brug
# wkt = 'POLYGON((117875.3 487304,117875.3 487329,117906.1 487329,117906.1 487304,117875.3 487304))' # brug

# stukje brug
# wkt = 'POLYGON((117890.1 487311,117890.1 487312.5,117892 487312.5,117892 487311,117890.1 487311))' # brug


xmin,ymin,xmax,ymax = shapely.wkt.loads(wkt).bounds

bounds = f'([{xmin}, {xmax}], [{ymin}, {ymax}])'
pc18 = get_points(path_a10_2018, bounds, wkt)
print(f'loaded pc 2018')

bounds = f'([{xmin}, {xmax}], [{ymin}, {ymax}])'
pc19 = get_points(path_a10_2019, bounds, wkt)

print(f'loaded pc 2019')

loaded pc 2018
loaded pc 2019


In [6]:
# from functions import filter_distance 

filtered_2019 = filter_distance(pc19, pc18)
print('filtered')
points_2019 = pd.DataFrame(
    np.array(filtered_2019[['X','Y','Z','Red','Green','Blue']].tolist()), 
    columns=['x', 'y', 'z','red','green','blue'])
print('dataframed')

# from functions import filter_distance 

filtered_2018 = filter_distance(pc18, pc19)
print('filtered')
points_2018 = pd.DataFrame(
    np.array(filtered_2018[['X','Y','Z','Red','Green','Blue']].tolist()), 
    columns=['x', 'y', 'z','red','green','blue'])
print('dataframed')

filtered
dataframed
filtered
dataframed


In [7]:
try:
    del points_with_planes
except:
    pass
planes18, models18 = recursive_planes(points_2018.copy(), 5, 200)
print('2018 done')
planes19, models19 = recursive_planes(points_2019.copy(), 5, 200)
print('2019 done')

clustering cluster 1
clustering cluster 2
clustering cluster 3
clustering cluster 4
clustering cluster 5
2018 done
clustering cluster 1
clustering cluster 2
clustering cluster 3
clustering cluster 4
clustering cluster 5
2019 done


In [8]:
rel_cids18 = get_relevant_cids(planes18)
rel_cids19 = get_relevant_cids(planes19)

In [9]:
for i in range(2):
    print('2018:', rel_cids18[i], models18[rel_cids18[i]].point, models18[rel_cids18[i]].normal)
    print('2019:', rel_cids19[i], models19[rel_cids19[i]].point, models19[rel_cids19[i]].normal)
    
    

2018: 1 [1.18046284e+05 4.88788266e+05 7.32619906e+00] [-0.01907583 -0.00597143  0.99980021]
2019: 1 [1.18043622e+05 4.88788105e+05 7.28090065e+00] [-0.0191627  -0.00399964  0.99980838]
2018: 2 [1.18067443e+05 4.88788088e+05 7.32255846e+00] [ 0.01882162 -0.00257657  0.99981954]
2019: 2 [1.18067129e+05 4.88788024e+05 7.33989558e+00] [-0.01913283  0.00412501 -0.99980844]


In [17]:
ransac = rfn.append_fields(filtered_2019[['X','Y','Z','Red','Green','Blue']], 
                              'Classification', 
                              planes19['cid'].astype(np.uint16),
                              usemask=False
                             )

write_to_laz(ransac, '/var/data/rws/data/output/tst_ransac/portal_planes.laz')
print('wrote')

wrote


In [13]:
models = {'2018: 1': [[1.18046284e+05, 4.88788266e+05, 7.32619906e+00], [-0.01907583, -0.00597143 , 0.99980021]],
'2019: 1': [[1.18043622e+05, 4.88788105e+05, 7.28090065e+00], [-0.0191627,  -0.00399964 , 0.99980838]],
'2018: 2': [[1.18067443e+05, 4.88788088e+05, 7.32255846e+00], [ 0.01882162 ,-0.00257657,0.99981954]],
'2019: 2': [[1.18067129e+05, 4.88788024e+05, 7.33989558e+00], [-0.01913283 , 0.00412501 ,-0.99980844]]}